<a href="https://colab.research.google.com/github/Gjeffroy/hyperparam_autotuning_keras/blob/main/NN_hyperparameter_tuning_witth_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install keras
! pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))  # Assuming input shape of (28, 28) for MNIST data

    # Tune the number of hidden layers
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load and preprocess the dataset
def load_and_prepare_mnist():
    # Load the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    # Normalize pixel values to between 0 and 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    # Reshape images to the format (batch_size, height, width, channels)
    train_images = train_images.reshape((-1, 28, 28, 1))
    test_images = test_images.reshape((-1, 28, 28, 1))

    # Convert labels to one-hot encoding
    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    return (train_images, train_labels), (test_images, test_labels)

# Load and prepare the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = load_and_prepare_mnist()

# Print shapes to verify
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Testing images shape:", test_images.shape)
print("Testing labels shape:", test_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
Training images shape: (60000, 28, 28, 1)
Training labels shape: (60000, 10)
Testing images shape: (10000, 28, 28, 1)
Testing labels shape: (10000, 10)


In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from kerastuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of hidden layers
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))

    # Output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Load and preprocess the dataset
def load_and_prepare_mnist():
    # Load the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    # Normalize pixel values to between 0 and 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    # Reshape images to the format (batch_size, height, width, channels)
    train_images = train_images.reshape((-1, 28, 28, 1))
    test_images = test_images.reshape((-1, 28, 28, 1))

    return (train_images, train_labels), (test_images, test_labels)

def main():
    # Load and prepare the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = load_and_prepare_mnist()

    # Initialize tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=3,
        directory='my_dir',
        project_name='mnist_tuning'
    )

    # Perform the hyperparameter search
    tuner.search(train_images, train_labels, epochs=5, validation_split=0.1)

    # Get the best model
    best_model = tuner.get_best_models(num_models=1)[0]

    # Train the best model on the full training dataset
    best_model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

    # Evaluate the best model on the test dataset
    loss, accuracy = best_model.evaluate(test_images, test_labels)
    print(f'Test accuracy: {accuracy}')

if __name__ == "__main__":
    main()

Trial 5 Complete [00h 02m 05s]
val_accuracy: 0.9671666423479716

Best val_accuracy So Far: 0.9756666620572408
Total elapsed time: 00h 14m 33s
Epoch 1/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0564 - accuracy: 0.9824 - val_loss: 0.0993 - val_accuracy: 0.9720
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0522 - accuracy: 0.9845 - val_loss: 0.1051 - val_accuracy: 0.9733
Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0464 - accuracy: 0.9858 - val_loss: 0.0814 - val_accuracy: 0.9777
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0425 - accuracy: 0.9875 - val_loss: 0.1020 - val_accuracy: 0.9760
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0379 - accuracy: 0.9888 - val_loss: 0.0867 - val_accuracy: 0.9788
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0360 - accuracy: 0.9891 - val_loss: 0.0885 - val_accuracy: 0

In [27]:
from kerastuner import HyperParameters
import os
import json

def summarize_tuner_attempts(directory):
    tuner_summaries = []

    # Iterate over each subdirectory in 'my_dir'
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Check if it contains a 'trial.json' file
            trial_file = os.path.join(subdir_path, 'trial.json')
            if os.path.exists(trial_file):
                # Load hyperparameters from 'trial.json'
                with open(trial_file, 'r') as f:
                    trial_data = json.load(f)
                hp = HyperParameters.from_config(trial_data['hyperparameters'])

                # Get the validation accuracy from 'trial.json'
                val_accuracy = trial_data.get('score')

                # Add hyperparameters and validation accuracy to the summaries list
                tuner_summaries.append((hp, val_accuracy))

    # Sort tuner summaries by the number of layers
    tuner_summaries.sort(key=lambda x: x[0].values['num_layers'])

    return tuner_summaries

# Function to print hyperparameters as a table
def print_hyperparameters_table(hp):
    print("Number of Layers:", hp.values['num_layers'])
    print("Learning Rate:", hp.values['learning_rate'])
    print("\nHyperparameters:")
    sorted_keys = sorted([key for key in hp.values.keys() if key.startswith('units')])
    for key in sorted_keys:
        print(f"| {key}: {hp.values[key]} |")
    print("\n")

# Function to print summary table
def print_summary_table(summary):
    print("Summary:")
    print("| Attempt | Accuracy | Num Layers |")
    print("|---------|----------|------------|")
    for i, (hp, val_accuracy) in enumerate(summary, 1):
        num_layers = hp.values['num_layers']
        print(f"| {i} | {val_accuracy} | {num_layers} |")
    print("\n")

# Example usage:
summaries = summarize_tuner_attempts('my_dir/mnist_tuning')
print_summary_table(summaries)
for i, (hp, val_accuracy) in enumerate(summaries, 1):
    print(f"Attempt {i}:")
    print_hyperparameters_table(hp)
    print(f"Validation Accuracy: {val_accuracy}\n")

Summary:
| Attempt | Accuracy | Num Layers |
|---------|----------|------------|
| 1 | 0.9671666423479716 | 2 |
| 2 | 0.958388884862264 | 4 |
| 3 | 0.9756666620572408 | 5 |
| 4 | 0.9728333155314127 | 6 |
| 5 | 0.9664999842643738 | 15 |


Attempt 1:
Number of Layers: 2
Learning Rate: 0.01

Hyperparameters:
| units_0: 320 |
| units_1: 256 |
| units_10: 64 |
| units_11: 288 |
| units_12: 320 |
| units_13: 384 |
| units_14: 320 |
| units_2: 192 |
| units_3: 384 |
| units_4: 288 |
| units_5: 256 |
| units_6: 32 |
| units_7: 192 |
| units_8: 32 |
| units_9: 96 |


Validation Accuracy: 0.9671666423479716

Attempt 2:
Number of Layers: 4
Learning Rate: 0.01

Hyperparameters:
| units_0: 448 |
| units_1: 224 |
| units_10: 320 |
| units_11: 512 |
| units_12: 224 |
| units_13: 160 |
| units_14: 128 |
| units_2: 352 |
| units_3: 320 |
| units_4: 288 |
| units_5: 416 |
| units_6: 256 |
| units_7: 416 |
| units_8: 64 |
| units_9: 128 |


Validation Accuracy: 0.958388884862264

Attempt 3:
Number of Lay